Presentación Final

Física Computacional - David Sanders


Miguel Raz-Guzmán Macedo

Modelo juguete de Ising para ferromagnetismo

![](http://cdn.instructables.com/F23/5RX1/GXL6FHS4/F235RX1GXL6FHS4.LARGE.jpg)

github.com/miguelraz/Projects/FinalSanders

I. Problema de interés
- ¿Cómo diantres funcionan los imanes?
- ¿Qué tipos de imanes hay y que los diferencia?

![](http://cdn.iopscience.com/images/0022-3727/42/3/032006/Full/jphysd292798fig03.jpg)

Nos encontramos con un problema de análisis de transición de fases de varios factores a nivel de estado sólido, y no tengo la más mínima idea de cómo pueden interaccionar de manera tan compleja.

Al agua patos - ¿Cómo lo modelo?

![](http://memesvault.com/wp-content/uploads/I-Have-No-Idea-What-Im-Doing-Dog-02.jpg)

II. Herramientas


Usaremos el modelo de Ising para representar los spines de los electrones en arreglo de superficie plano representado 1 y -1s.

Actualizaremos puntos arbitrarios dadas las condiciones iniciales y simplificaciones del problema y graficaremos los comportamientos correspondientes. 

Primero definiremos varias funciones para embonar y construir nuestra simulación completa.

In [ ]:
using PyPlot
Ntiempo = 10000
NGridSize = 5 #Arista de cubo
T = 370 # En Kelvin
κ = 1 # Constante de intercambio de energia
β = 1/(κ * T)
J = 1 #Cte de Magnetizacion

In [21]:
#Crear arreglo que nos interesa
function RestartGrid(NGridSize=10)
    Grid = rand(-1:2:1,NGridSize,NGridSize)
    return Grid
end

In [ ]:
RestartGrid()

In [ ]:
function RestartChoice(NGridSize=10)
    
    Row = rand(1:NGridSize)
    Col = rand(1:NGridSize)
    return Row, Col
end

In [ ]:
RestartChoice()

In [ ]:
# Estableciendo por separado puntos de referencua
YSouth = mod(Row,NGridSize)+1
XWest = mod(Col,  NGridSize) +1
XEast = mod(Col-2,  NGridSize) +1
YNorth = mod(Row-2, NGridSize) +1

#Por favor, investiguen la funcion mod1()

In [ ]:
A = reshape(1:NGridSize^2,NGridSize,NGridSize)

In [ ]:
#Test!
Row = 3
Col = 2
A[YNorth,Col],
A[YSouth, Col],
A[Row, XWest],
A[Row, XEast]

In [ ]:
# Datos procesados del sistema en un determinado instante

North(Row,Col) = J*(Grid[Row,Col] * Grid[YNorth, Col])

East(Row,Col) = J*(Grid[Row,Col] * Grid[Row, XEast])

West(Row,Col) = J*(Grid[Row,Col] * Grid[Row, XWest])

South(Row,Col) = J*(Grid[Row,Col] * Grid[YSouth, Col])

NESW(Row,Col) = North(Row,Col) + East(Row,Col) + South(Row,Col) + West(Row,Col) 

In [ ]:
function Energia_σ(Row,Col) 
    energia = 0
    energia = -1 * NESW(Row,Col)
    return energia
end

In [ ]:
Grid = RestartGrid()

In [ ]:
#Sanity Test! Queremos comprobar que nuestras regiones fantasmas no se rompan.

NGridSize = 10
Grid = RestartGrid()

Row, Col = RestartChoice()

North(Row, Col), East(Row, Col), South(Row, Col), West(Row, Col), Energia_σ(Row, Col)

In [ ]:
function EnergiaTotalN()
    energiaTotalN = 0
    
    for Row in 1:NGridSize, Col in 1:NGridSize
        
        energiaTotalN = energiaTotalN + Energia_σ(Row,Col)              
    end
    
    return energiaTotalN *.5 # Interesante! Cada energia la calculo 2 veces. Hay que descontar.
end   

In [ ]:
Grid = RestartGrid()
EnergiaTotalN()

In [ ]:
function Magnetizacion(Grid)
    magnetizacion = sum(Grid)
    return magnetizacion
end    

In [ ]:
Magnetizacion(Grid)

In [ ]:
function Flipper(Row,Col,Energias,EnergiaTotalN,energiaTotalN,FlipCounter,Flips,Magnetismos,magnetizacion)

    proba_σ = 0
    energia_σ = 0
    
    proba_σ = Proba_σ(Row,Col)
    energia_σ = Energia_σ(Row,Col)
    
    if rand() < proba_σ
        
        Grid[Row,Col] *= -1
        
        energia_s = Energia_σ(Row,Col)
        
        energiaTotalN = energiaTotalN + 2*energia_σ # puede que vaya a la mitad

        FlipCounter = FlipCounter + 1
        
        magnetizacion = magnetizacion + 2*Grid[Row,Col]
    end 
    
        push!(Energias, energiaTotalN)
        push!(Flips, FlipCounter)
        push!(Magnetismos, magnetizacion)
    
    return energiaTotalN, FlipCounter, magnetizacion
end

In [ ]:
Row = 2
Col = 2
Energias = []
energiaTotalN = 2
Magnetismos = []
magnetizacion = 2



In [ ]:
Magnetismos

In [ ]:
energiaTotalN, FlipCounter, magnetizacion = Flipper(Row,Col,EnergiaTotalN,energiaTotalN,FlipCounter,Flips,Magnetismos, magnetizacion)

Hecha la talacha, llegamos a nuestro gran ciclo de simulación.

In [ ]:
# Funcion de calculo para toda la energia del sistema en determinado tiempo
function Simulation(T,J,NGridSize.Ntiempo=100) 

    Tiempos = []
    Energias = []
    Flips = []
    FlipCounter = 0
    Magnetismos = []
    
    
    #Arrancando motores
    Grid = RestartGrid(NGridSize)
    energiaTotalN = EnergiaTotalN()
    magnetizacion = Magnetizacion(Grid)

    # Loop principal // Let the magic begin!
    for t in 1:Ntiempo    
        
        Row,Col = RestartChoice()
        energiaTotalN, FlipCounter, magnetizacion = Flipper(Row,Col,Energias, EnergiaTotalN,energiaTotalN,FlipCounter,Flips,Magnetismos, magnetizacion)
        push!(Tiempos,t)
    end 
    println(length(Flips), " Flips")
    println(length(Magnetismos), " Magnetismos")
    println(length(Energias), " Energias")
    println(length(Tiempos), " Tiempos")
    plot(Tiempos, Flips)
    plot(Tiempos, Magnetismos)
    plot(Tiempos, Energias)
    
end

In [ ]:
Simulation()

In [ ]:
# Como buenos físicos, si ya tenemos nuevo juguetito, queda ver hasta dónde se puede estirar sus capacidades...
# A ver si podemos encontrar los 3 tipos de magnetismo clásico...
T = 300
J = 1 
NGridSize = 100
NTiempo = 100

Simulation(T,J,NGridSize,NTiempo)

In [ ]:
Conclusiones del comportamiento físico:
- Tenemos una transición de fase MUY abrupta para estados que no están previamente magnetizados
- Existe una temperatura después de la cual no se observa más magnetización



In [ ]:
Debilidades del modelo y futuras áreas de interés:
- Teselación hexagonal en 2D = más resolución
- Spin en 360 grados
- Modelos de interacción a mayor orden (No sólo vecinos contiguos)
- Comportamientos bajo gradientes de temperatura

In [23]:
Quejas dudas comentarios sugerencias aclaraciones mentadas facepalms

... pull request!

LoadError: LoadError: syntax: extra token "dudas" after end of expression
while loading In[23], in expression starting on line 1